### NOTE: for WP basin have to run on AWS as encounter memory issues

In [1]:
import os

import pandas as pd
import rioxarray
from rasterio.errors import RasterioIOError
import numpy as np
import dask

In [2]:
# User specified inputs
landcover_raster = '/home/mark/projects/open-gira/results/direct/WP/GlobCover.tif'
wind_grid = '/home/mark/projects/open-gira/results/direct/WP/wind_grid.tiff'
land_cover_roughness_mapping = '/home/mark/projects/open-gira/config/land_use_to_surface_roughness.csv'
output = '/home/mark/projects/open-gira/results/direct/WP/surface_roughness.tif'

In [ ]:
try:
    land_cover = rioxarray.open_rasterio(landcover_raster)
except RasterioIOError:
    logging.info("Found empty land cover map, creating empty surface roughness raster...")
    os.system(f"touch {output}")
    sys.exit(0)
wind_grid = rioxarray.open_rasterio(wind_grid)
cover_roughness = pd.read_csv(land_cover_roughness_mapping, comment="#")

# build a lookup array where the category is the index
# and the value for a given index/category is the surface roughness in metres
lookup_length = max(set(cover_roughness.glob_cover_2009_id)) + 1
roughness_lookup: np.ndarray = np.zeros(lookup_length)
for row in cover_roughness.itertuples():
    roughness_lookup[row.glob_cover_2009_id] = row.roughness_length_m

# create surface roughness field (same shape as land cover)
surface_roughness_values: np.ndarray = roughness_lookup[land_cover.values]
surface_roughness = land_cover.copy()
surface_roughness.values = surface_roughness_values

# surface roughness on wind grid
downsampled_roughness = surface_roughness.interp(x=wind_grid.x, y=wind_grid.y)

In [ ]:
# Save
downsampled_roughness.rio.to_raster(output)